In [36]:
#How does financial literacy vary across age groups and genders in the 2024 NFCS data?
#How does financial literacy vary across regions and divisions in the 2024 NFCS data?

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# === 1) Load original dataset ===
cleaned_df = pd.read_csv("2012 Cleaned.csv", na_values=["", " ", "NA", "N/A", "null", ".", "na"])
original_df = pd.read_csv("NFCS 2012 State Data 130503.csv", na_values=["", " ", "NA", "N/A", "null", ".", "na"])

# === NFCS 2012 — value label dictionaries (from the PDF) ===

STATEQ = {
    1:"Alabama",2:"Alaska",3:"Arizona",4:"Arkansas",5:"California",6:"Colorado",7:"Connecticut",8:"Delaware",9:"District of Columbia",
    10:"Florida",11:"Georgia",12:"Hawaii",13:"Idaho",14:"Illinois",15:"Indiana",16:"Iowa",17:"Kansas",18:"Kentucky",19:"Louisiana",
    20:"Maine",21:"Maryland",22:"Massachusetts",23:"Michigan",24:"Minnesota",25:"Mississippi",26:"Missouri",27:"Montana",
    28:"Nebraska",29:"Nevada",30:"New Hampshire",31:"New Jersey",32:"New Mexico",33:"New York",34:"North Carolina",35:"North Dakota",
    36:"Ohio",37:"Oklahoma",38:"Oregon",39:"Pennsylvania",40:"Rhode Island",41:"South Carolina",42:"South Dakota",43:"Tennessee",
    44:"Texas",45:"Utah",46:"Vermont",47:"Virginia",48:"Washington",49:"West Virginia",50:"Wisconsin",51:"Wyoming"
}

CENSUSDIV = {
    1:"New England",2:"Middle Atlantic",3:"East North Central",4:"West North Central",
    5:"South Atlantic",6:"East South Central",7:"West South Central",8:"Mountain",9:"Pacific"
}

CENSUSREG = {1:"Northeast",2:"Midwest",3:"South",4:"West"}

A3 = {1:"Male",2:"Female"}

A3Ar_w = {1:"18-24",2:"25-34",3:"35-44",4:"45-54",5:"55-64",6:"65+"}

A3B = {
    1:"Male 18-24",2:"Male 25-34",3:"Male 35-44",4:"Male 45-54",5:"Male 55-64",6:"Male 65+",
    7:"Female 18-24",8:"Female 25-34",9:"Female 35-44",10:"Female 45-54",11:"Female 55-64",12:"Female 65+"
}

A4A_new_w = {1:"While Alone NH",2:"Non-White"}  # (verbatim label in PDF)

A5_2012 = {
    1:"Did not complete high school",
    2:"High school graduate - regular high school diploma",
    3:"High school graduate - GED or alternative credential",
    4:"Some college",
    5:"College graduate",
    6:"Post graduate education",
    99:"Prefer not to say"
}

A6 = {1:"Married",2:"Single",3:"Separated",4:"Divorced",5:"Widowed/widower",99:"Prefer not to say"}

A7 = {
    1:"I am the only adult in the household",
    2:"I live with my spouse/partner/significant other",
    3:"I live in my parents' home",
    4:"I live with other family, friends, or roommates",
    99:"Prefer not to say"
}

A7A = {1:"Married",2:"Living with partner",3:"Single"}

A11 = {
    1:"1",2:"2",3:"3",4:"4 or more",5:"No financially dependent children",6:"Do not have any children",99:"Prefer not to say"
}

A8 = {
    1:"Less than $15,000",
    2:"At least $15,000 but less than $25,000",
    3:"At least $25,000 but less than $35,000",
    4:"At least $35,000 but less than $50,000",
    5:"At least $50,000 but less than $75,000",
    6:"At least $75,000 but less than $100,000",
    7:"At least $100,000 but less than $150,000",
    8:"$150,000 or more",
    98:"Don't know",99:"Prefer not to say"
}

AM21 = {1:"Currently a member of the U.S. Armed Services",2:"Previously a member of the U.S. Armed Services",3:"Never a member of the U.S. Armed Services",99:"Prefer not to say"}

AM22 = {1:"Currently a member of the U.S. Armed Services",2:"Previously a member of the U.S. Armed Services",3:"Never a member of the U.S. Armed Services",99:"Prefer not to say"}

X3 = {1:"Core questions",2:"Military"}

X4 = {1:"Respondent in service",2:"Spouse in service"}

AM8 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

A9 = {
    1:"Self employed",2:"Work full-time for an employer [or the military]",3:"Work part-time for an employer [or the military]",
    4:"Homemaker",5:"Full-time student",6:"Permanently sick, disabled, or unable to work",
    7:"Unemployed or temporarily laid off",8:"Retired",99:"Prefer not to say"
}

A10 = {
    1:"Self employed",2:"Work full-time for an employer [or the military]",3:"Work part-time for an employer [or the military]",
    4:"Homemaker",5:"Full-time student",6:"Permanently sick, disabled, or unable to work",7:"Unemployed or temporarily laid off",8:"Retired",99:"Prefer not to say"
}

A10A = {1:"Non-retired household",2:"Retired household--Respondent retired",3:"Retired household--Respondent not working and spouse retired"}

A21 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

A22 = {1:"Four-year college or university",2:"Two-year community college",3:"Vocational, technical, or trade school",4:"Other",98:"Don't know",99:"Prefer not to say"}

A14 = {1:"You",2:"Someone else",3:"You and someone else are equally knowledgeable",98:"Don't know",99:"Prefer not to say"}

J1 = {**{i:str(i) for i in range(1,10)}, 1:"1 - Not At All Satisfied",10:"10 - Extremely Satisfied",98:"Don't know",99:"Prefer not to say"}

J2 = {**{i:str(i) for i in range(2,10)}, 1:"1 - Not At All Willing",10:"10 - Very Willing",98:"Don't know",99:"Prefer not to say"}

J3 = {1:"Spending less than income",2:"Spending more than income",3:"Spending about equal to income",98:"Don't know",99:"Prefer not to say"}

J4 = {1:"Very difficult",2:"Somewhat difficult",3:"Not at all difficult",98:"Don't know",99:"Prefer not to say"}

J5 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

J6 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

J8 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

J9 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

J10 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

J20 = {
    1:"I am certain I could come up with the full $2,000",
    2:"I could probably come up with $2,000",
    3:"I could probably not come up with $2,000",
    4:"I am certain I could not come up with $2,000",
    98:"Don't know",99:"Prefer not to say"
}

J11 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}
J12 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

K_1 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}
K_2 = K_1.copy()
K_3 = K_1.copy()
K_4 = K_1.copy()
K_5 = K_1.copy()

B20_1 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}
B20_2 = B20_1.copy()
B20_3 = B20_1.copy()
B20_4 = B20_1.copy()

B11A = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}

B21 = {1:"Cash",2:"Checks (i.e., paper checks that need to be deposited or cashed)",3:"Direct deposit to a checking account or savings account",4:"Prepaid debit cards (e.g., a payroll card from an employer, or alimony or unemployment pay)",98:"Don't know",99:"Prefer not to say"}

B22_1 = {1:"Frequently",2:"Sometimes",3:"Never",98:"Don't know",99:"Prefer not to say"}
B22_2 = B22_1.copy()
B22_3 = B22_1.copy()
B22_4 = B22_1.copy()
B22_5 = B22_1.copy()
B22_6 = B22_1.copy()
B22_7 = B22_1.copy()
B22_8 = B22_1.copy()

B1 = {1:"Yes",2:"No",98:"Don't know",99:"Prefer not to say"}
B2 = B1.copy()
B4 = B1.copy()
B23 = B1.copy()
B14 = B1.copy()

C1_2012 = B1.copy()
C2_2012 = {1:"Your employer",2:"Your [spouse's/partner's] employer",3:"Both your employer and your [spouse's/partner's] employer",98:"Don't know",99:"Prefer not to say"}
C3_2012 = B1.copy()
C4_2012 = B1.copy()
C5_2012 = B1.copy()
C10_2012 = B1.copy()
C11_2012 = B1.copy()

D20_1 = B1.copy()
D20_2 = B1.copy()
D20_3 = B1.copy()
D20_4 = B1.copy()
D20_5 = B1.copy()
D20_6 = B1.copy()
D20_7 = B1.copy()

EA_1 = B1.copy()
EA_2A = B1.copy()

E4A = {
    1:"1999 or earlier",2:"2000",3:"2001",4:"2002",5:"2003",6:"2004",7:"2005",8:"2006",
    9:"2007",10:"2008",11:"2009",12:"2010",13:"2011",14:"2012",97:"You did not purchase it",98:"Don't know",99:"Prefer not to say"
}

E5_2012 = {-99:"Prefer not to say",-98:"Don't know"}  # percentage entered as numeric otherwise

E7 = B1.copy()
E8 = B1.copy()

E20 = {1:"Yes, owe more",2:"No",98:"Don't know",99:"Prefer not to say"}

E15 = {1:"Never",2:"Once",3:"More than once",98:"Don't know",99:"Prefer not to say"}

E16 = B1.copy()

F1 = {
    1:"1",2:"2-3",3:"4-8",4:"9-12",5:"13-20",6:"More than 20",7:"No credit cards",98:"Don't know",99:"Prefer not to say"
}

F2_1 = B1.copy()
F2_2 = B1.copy()
F2_3 = B1.copy()
F2_4 = B1.copy()
F2_5 = B1.copy()
F2_6 = B1.copy()

F10 = B1.copy()

G1 = B1.copy()
G20 = B1.copy()
G21 = B1.copy()
G22 = B1.copy()
G4  = B1.copy()

G25_1 = {1:"Never",2:"1 time",3:"2 times",4:"3 times",5:"4 or more times",98:"Don't know",99:"Prefer not to say"}
G25_2 = G25_1.copy()
G25_3 = G25_1.copy()
G25_4 = G25_1.copy()
G25_5 = G25_1.copy()

G23 = {1:"1 - Strongly Disagree",2:"2",3:"3",4:"4 - Neither Agree nor Disagree",5:"5",6:"6",7:"7 - Strongly Agree",98:"Don't know",99:"Prefer not to say"}

H1 = B1.copy()
H3 = B1.copy()

M1_1 = G23.copy()
M1_2 = G23.copy()

M4 = {1:"1 - Very Low",2:"2",3:"3",4:"4",5:"5",6:"6",7:"7 - Very High",98:"Don't know",99:"Prefer not to say"}

M20 = {1:"Yes, but I did not participate in the financial education",2:"Yes, and I did participate in the financial education",3:"No",98:"Don't know",99:"Prefer not to say"}

M21_1 = B1.copy()
M21_2 = B1.copy()
M21_3 = B1.copy()
M21_4 = B1.copy()

M22 = B1.copy()

M6 = {1:"More than $102",2:"Exactly $102",3:"Less than $102",98:"Don't know",99:"Prefer not to say"}
M7 = {1:"More than today",2:"Exactly the same",3:"Less than today",98:"Don't know",99:"Prefer not to say"}
M8 = {1:"They will rise",2:"They will fall",3:"They will stay the same",4:"There is no relationship between bond prices and the interest rate",98:"Don't know",99:"Prefer not to say"}

M9 = {1:"True",2:"False",98:"Don't know",99:"Prefer not to say"}
M10 = M9.copy()

# Weights (continuous)
wgt_n2 = None
wgt_d2 = None
wgt_s3 = None

# === Column -> dictionary mapping ===
LABELS_BY_COLUMN = {
    # IDs & Geography
    "STATEQ": STATEQ,
    "CENSUSDIV": CENSUSDIV,
    "CENSUSREG": CENSUSREG,

    # Demographics
    "A3": A3, "A3Ar_w": A3Ar_w, "A3B": A3B, "A4A_new_w": A4A_new_w, "A5_2012": A5_2012,
    "A6": A6, "A7": A7, "A7A": A7A, "A11": A11, "A8": A8,
    "AM21": AM21, "AM22": AM22, "X3": X3, "X4": X4, "AM8": AM8,

    # Employment & schooling
    "A9": A9, "A10": A10, "A10A": A10A, "A21": A21, "A22": A22, "A14": A14,

    # Household finances & behaviors
    "J1": J1, "J2": J2, "J3": J3, "J4": J4, "J5": J5, "J6": J6, "J8": J8, "J9": J9,
    "J10": J10, "J20": J20, "J11": J11, "J12": J12,

    # Advice
    "K_1": K_1, "K_2": K_2, "K_3": K_3, "K_4": K_4, "K_5": K_5,

    # Income methods & payments
    "B20_1": B20_1, "B20_2": B20_2, "B20_3": B20_3, "B20_4": B20_4,
    "B11A": B11A, "B21": B21,
    "B22_1": B22_1, "B22_2": B22_2, "B22_3": B22_3, "B22_4": B22_4,
    "B22_5": B22_5, "B22_6": B22_6, "B22_7": B22_7, "B22_8": B22_8,

    # Accounts & investments
    "B1": B1, "B2": B2, "B4": B4, "B23": B23, "B14": B14,

    # Retirement
    "C1_2012": C1_2012, "C2_2012": C2_2012, "C3_2012": C3_2012, "C4_2012": C4_2012,
    "C5_2012": C5_2012, "C10_2012": C10_2012, "C11_2012": C11_2012,

    # Income sources (past 12 months)
    "D20_1": D20_1, "D20_2": D20_2, "D20_3": D20_3, "D20_4": D20_4,
    "D20_5": D20_5, "D20_6": D20_6, "D20_7": D20_7,

    # Housing
    "EA_1": EA_1, "EA_2A": EA_2A, "E4A": E4A, "E5_2012": E5_2012, "E7": E7, "E8": E8, "E20": E20, "E15": E15, "E16": E16,

    # Credit & debt
    "F1": F1, "F2_1": F2_1, "F2_2": F2_2, "F2_3": F2_3, "F2_4": F2_4, "F2_5": F2_5, "F2_6": F2_6, "F10": F10,
    "G1": G1, "G20": G20, "G21": G21, "G22": G22, "G4": G4,
    "G25_1": G25_1, "G25_2": G25_2, "G25_3": G25_3, "G25_4": G25_4, "G25_5": G25_5, "G23": G23,

    # Insurance
    "H1": H1, "H3": H3,

    # Confidence / knowledge / education
    "M1_1": M1_1, "M1_2": M1_2, "M4": M4, "M20": M20,
    "M21_1": M21_1, "M21_2": M21_2, "M21_3": M21_3, "M21_4": M21_4,
    "M22": M22, "M6": M6, "M7": M7, "M8": M8, "M9": M9, "M10": M10,

    # Weights (continuous/no mapping)
    "wgt_n2": None, "wgt_d2": None, "wgt_s3": None
}


# Financial Literacy

### Financial Literacy is defined by the following columns
"J5": "Emergency Funds",
"J6": "College Savings",
"J20": "Emergency Confidence",
"J32": "Credit Record",
"B1": "Checking Account",
"B2": "Savings Account",
"B4": "Overdraw",
"C1_2012": "Retirement Plan",
"C5_2012": "Other Retirement Accounts",
"B14A_1": "Other Investments",
"E15_2015": "Mortgage Payment",
"F1": "Credit Card",
"G20": "Past Due Payments",
"G35": "Late Student Loan",
"G38": "Debt Agency Contact",
"M1_1": "Financial Confidence",
"M4": "Financial Knowledge"

### To score a person on financial literacy we will be scoring them weighting their scores based on the format of the questions 
(To discuss with group)
Current Scheme:
J5: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
J6: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
J20: Certain could come up with full $2,000 (5), Probably could (4), Probably could not (3), certain could not (2), Don't know (0), Prefer not to say (0), No Response Given (0)
J32: Very Good (5), Good (4), About Average (3), Bad (2), Very Bad (1), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
B1: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
B2: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
B4: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
C1_2012: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0) 
C5_2012: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
B14A_1: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0) 
E15_2015: Never (?), Once (?), More than once (?), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
F1: 2 to 3 (?), 3 to 4 (?), 1 (?), 4 to 8 (?), 9 to 12 (?), 13 to 20 (?), More than 20, No credit cards (?), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
G20: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0) 
G35: Never, payments not due (?), Never (?), repaying on time each month (?), Once (?), More than once (?), Don't Know (0), Prefer Not to Say (0), No Response Given (0)
G38: Yes (3), No (2), Don't Know (0), Prefer Not to Say (0), No Response Given (0)


In [37]:
# ----- Bucket 1: Critical Stability -----
J5_w = {1: 5, 2: 1, 98: 0, 99: 0, "No Response Given": 0}  # Emergency fund
J20_w = {1: 5, 2: 3, 3: 2, 4: 1, 98: 0, 99: 0, "No Response Given": 0}  # $2k confidence
E15_w = {1: 5, 2: 3, 3: 1, 98: 0, 99: 0, "No Response Given": 0}  # Mortgage late
G38_w = {2: 5, 1: 0, 98: 0, 99: 0, "No Response Given": 0}  # Debt collector contact

# ----- Bucket 2: Core Access & Assets -----
B1_w = {1: 3, 2: 1, 98: 0, 99: 0, "No Response Given": 0}  # Checking account
B2_w = {1: 3, 2: 1, 98: 0, 99: 0, "No Response Given": 0}  # Savings account
C1_2012_w = {1: 3, 2: 1, 98: 0, 99: 0, "No Response Given": 0}  # Any retirement plan
C5_2012_w = {1: 3, 2: 1, 98: 0, 99: 0, "No Response Given": 0}  # Contribute to retirement
B14_w = {1: 3, 2: 1, 98: 0, 99: 0, "No Response Given": 0}  # Non-retirement investments

# ----- Bucket 3: Credit Health & Debt Behavior -----
F1_w = {2: 3, 3: 3, 1: 2, 4: 1, 5: 0, 6: 0, 7: 1, 98: 0, 99: 0, "No Response Given": 0}  # Credit cards
B4_w = {2: 3, 1: 1, 98: 0, 99: 0, "No Response Given": 0}  # Overdraft
G20_w = {2: 3, 1: 1, 98: 0, 99: 0, "No Response Given": 0}  # Past-due medical bills

# ----- Bucket 4: Supplementary Planning -----
J6_w = {1: 2, 2: 1, 98: 0, 99: 0, "No Response Given": 0}  # College savings

WEIGHTS = {
    "Emergency Funds": J5_w,
    "College Savings": J6_w,
    "Emergency Confidence": J20_w,
    "Checking Account": B1_w,
    "Savings Account": B2_w,
    "Overdraw": B4_w,
    "Retirement Plan": C1_2012_w,
    "Other Retirement Accounts": C5_2012_w,
    "Other Investments": B14_w,
    "Mortgage Payment": E15_w,
    "Credit Card": F1_w,
    "Past Due Payments": G20_w,
}


In [38]:
# === 3) Sorting Variables ===

# Rename dictionary for predictors
predictor_rename_dict = {
    "A5_2015": "Education",
    "A11": "Dependent Children",
    "A8_2021": "Annual Income",
    "A50A": "Gender",
    "A3Ar_w": "Age Group",
    "A50B": "Gender&Age"
}

# Rename dictionary for financial literacy outcomes
finlit_rename_dict = {
    "J5": "Emergency Funds",
    "J6": "College Savings",
    "J20": "Emergency Confidence",
    "B1": "Checking Account",
    "B2": "Savings Account",
    "B4": "Overdraw",
    "C1_2012": "Retirement Plan",
    "C5_2012": "Other Retirement Accounts",
    "B14": "Other Investments",          
    "E15": "Mortgage Payment",          
    "F1": "Credit Card",
    "G20": "Past Due Payments",
    "M1_1": "Financial Confidence",
    "M4": "Financial Knowledge"
}


# Combine both into one dictionary
rename_dict = {**predictor_rename_dict, **finlit_rename_dict}

# Rename columns in dataframe
original_df = original_df.rename(columns=rename_dict)

In [39]:
# Defining Financial Literacy
financial_literacy = original_df.copy()
print(financial_literacy.head(10).to_string())

for col, weight_dict in WEIGHTS.items():
    # financial_literacy[f"{col}_score"] = financial_literacy[col].map(weight_dict)
    original_df[f"{col}_score"] = original_df[col].map(weight_dict)



       NFCSID  STATEQ  CENSUSDIV  CENSUSREG  A3  Age Group  A3B  A4A_new_w  A5_2012  A6  A7  A7A  Dependent Children  A8  AM21  AM22  X3  X4  AM8  A9  A10  A10A  A21  A22  A14  J1  J2  J3  J4  Emergency Funds  College Savings    J8    J9  J10  Emergency Confidence  J11  J12  K_1  K_2  K_3  K_4  K_5  B20_1  B20_2  B20_3  B20_4  B11A  B21  B22_1  B22_2  B22_3  B22_4  B22_5  B22_6  B22_7  B22_8  Checking Account  Savings Account  Overdraw  B23  Other Investments  Retirement Plan  C2_2012  C3_2012  C4_2012  Other Retirement Accounts  C10_2012  C11_2012  D20_1  D20_2  D20_3  D20_4  D20_5  D20_6  D20_7  EA_1  EA_2A   E4A  E5_2012   E7   E8   E20  Mortgage Payment  E16  Credit Card  F2_1  F2_2  F2_3  F2_4  F2_5  F2_6  F10  G1  Past Due Payments  G21  G22  G4  G25_1  G25_2  G25_3  G25_4  G25_5  G23  H1  H3  Financial Confidence  M1_2  Financial Knowledge  M20  M21_1  M21_2  M21_3  M21_4  M22  M6  M7  M8  M9  M10    wgt_n2    wgt_d2    wgt_s3
0  2012010001      24          4          2   2     

In [40]:
original_df["Total_Score"] = original_df[[f"{col}_score" for col in WEIGHTS.keys()]].sum(axis=1)

conditions = [
    (original_df["Total_Score"] >= 49),
    (original_df["Total_Score"] >= 41) & (original_df["Total_Score"] < 49),
    (original_df["Total_Score"] >= 27) & (original_df["Total_Score"] < 41),
    (original_df["Total_Score"] >= 18) & (original_df["Total_Score"] < 27),
    (original_df["Total_Score"] >= 9) & (original_df["Total_Score"] < 18),
    (original_df["Total_Score"] >= 1) & (original_df["Total_Score"] < 9),
    (original_df["Total_Score"] == 0)
]


choices = ["Perfect", "High", "Average", "Below Average", "Low", "Bad", "None"]

original_df["FinLit_Level"] = np.select(conditions, choices, default="No Response Given")



In [41]:
print("Pre-mapping", original_df.isna().sum().sum())

for col, mapping in LABELS_BY_COLUMN.items():
    if col in original_df.columns and mapping is not None:
        original_df[col] = original_df[col].map(mapping)

print("Post-mapping", cleaned_df.isna().sum().sum())

original_df.fillna("No Response Given", inplace=True)

print("Post-fillna", cleaned_df.isna().sum().sum())


Pre-mapping 546246
Post-mapping 48
Post-fillna 48


C:\Users\Academic\AppData\Local\Temp\ipykernel_82492\638833436.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No Response Given' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  original_df.fillna("No Response Given", inplace=True)


In [42]:


original_df.to_csv("2012 Cleaned.csv", index=False)
# Define new lists with human-readable names
predictors = list(predictor_rename_dict.values())
financial_lit_cols = list(finlit_rename_dict.values())

print("Post-cleaning", original_df.isna().sum().sum())

Post-cleaning 0
